## Question 3

#### Imports

In [1]:
import itertools
import os
from datetime import datetime

import numpy as np

from classify import temperature
from classify.data.responses import responses_to_traffic_array
from classify.data.traffic import load_traffic
from classify.scenario.bridge import healthy_damage, pier_disp_damage
from classify.scenario.traffic import normal_traffic
from fem.responses import load_fem_responses
from fem.run.opensees import OSRunner
from model.bridge import Point
from model.bridge.bridge_705 import bridge_705_3d, bridge_705_med_config
from model.response import ResponseType
from util import resize_units

#### Config

In [2]:
DATA_PATH = "/Users/jeremy/Desktop/mesh-med-600-mar/"
IMAGE_PATH = "/Users/jeremy/Desktop/saved-images/"
ERROR_TERM = True
c = bridge_705_med_config(bridge_705_3d)
# Set the directory of where to save/load responses.
c.root_generated_data_dir = os.path.join(DATA_PATH, c.root_generated_data_dir)

INFO: Loaded vehicle data from /Users/jeremy/cs/bridge-dss/data/a16-data/a16.csv in 0.10s
WARN: Vehicle PDF sums to 99.5, adjusted to sum to 1


#### Aliases and sensors

In [3]:
# Short aliases for response types.
rt_y = ResponseType.YTranslation
rt_s = ResponseType.Strain

# Create functions to resize, and unit strings, for each response type.
resize_y, units_y = resize_units(rt_y.units())
resize_s, _ = resize_units(rt_s.units())

sensor_point = Point(x=21, y=0, z=-8.4)  # A sensor point to investigate.
sensor_point = Point(x=33, y=0, z=-4)  # A sensor point to investigate.

#### Traffic data

In [4]:
total_mins = 24
total_seconds = total_mins * 60
traffic_scenario = normal_traffic(c=c, lam=5, min_d=2)
traffic_sequence_0, traffic_0, traffic_array_0 = load_traffic(
    c=c,
    traffic_scenario=traffic_scenario,
    max_time=total_seconds,
)

/Users/jeremy/Desktop/mesh-med-600-mar/generated-data/bridge-705-3d/healthy/traffic/normal-lam-5-600-1440-0,01.npy


#### 2019 temperature data

In [5]:
# Load minute-by-minute temperature data for the year.
from_ = datetime.fromisoformat(f"2019-01-01T00:00")
to = datetime.fromisoformat(f"2019-12-31T23:59")
temps_2019 = temperature.from_to_mins(temperature.load("holly-springs"), from_, to)
temps_2019["temp"] = temperature.resize(temps_2019["temp"])

Temperature (0.001, 0.999) quantiles = -9.5, 34.5


In [6]:
# Effect of annual temperature.
effect_2019_y = resize_y(temperature.effect(c=c, response_type=rt_y, points=[sensor_point], temps=temps_2019["temp"], solar=temps_2019["solar"], len_per_hour=60)[0])
effect_2019_s = temperature.effect(c=c, response_type=rt_s, points=[sensor_point], temps=temps_2019["temp"], solar=temps_2019["solar"], len_per_hour=60)[0]
assert len(temps_2019) == len(effect_2019_y)

#### Response to 2mm pier settlement

In [7]:
error_scenario = pier_disp_damage([(5, 2 / 1000)])
error_response = float(load_fem_responses(
    c=error_scenario.use(c)[0],
    sim_params=error_scenario.use(c)[1],
    response_type=ResponseType.YTranslation,
    sim_runner=OSRunner,
).at_deck(point=sensor_point, interp=True))
error_response

-0.0014309792947987657

#### 2018 temperature data

In [8]:
# Load minute-by-minute temperature data for the year.
from_ = datetime.fromisoformat(f"2018-01-01T00:00")
to = datetime.fromisoformat(f"2018-12-31T23:59")
temps_2018 = temperature.from_to_mins(temperature.load("holly-springs-18"), from_, to)
temps_2018["temp"] = temperature.resize(temps_2018["temp"])

Temperature (0.001, 0.999) quantiles = -16.6881, 33.5


In [9]:
# Effect of annual temperature.
effect_2018_y = resize_y(temperature.effect(c=c, response_type=rt_y, points=[sensor_point], temps=temps_2018["temp"], solar=temps_2018["solar"], len_per_hour=60)[0])
print(np.min(effect_2018_y), effect_2018_y.shape)
effect_2018_s = temperature.effect(c=c, response_type=rt_s, points=[sensor_point], temps=temps_2018["temp"], solar=temps_2018["solar"], len_per_hour=60)[0]
assert len(temps_2018) == len(effect_2018_y)
if ERROR_TERM:
    error_y = resize_y(error_response)
    effect_2018_y = effect_2018_y + np.interp(np.arange(len(effect_2018_y)), [0, len(effect_2018_y) - 1], [0, error_y])
    print(np.min(effect_2018_y), effect_2018_y.shape)

-1.1728296313840254MResponses in 0.01s, (ResponseType.YTranslation) (525600,)
-2.1482168474616006MResponses in 0.01s, (ResponseType.Strain) (525600,)


#### Regression of 2018 temperature data

In [10]:
daily_samples = 60 * 24
num_days_2018 = len(temps_2018) / daily_samples
assert num_days_2018 == 365

regress_temps_2018 = []
regress_y_2018 = []
regress_s_2018 = []
regress_dates_2018 = []
for i in range(int(num_days_2018)):
    lo = i * daily_samples
    hi = lo + daily_samples
    regress_temps_2018.append(np.median(temps_2018["temp"][lo:hi]))
    regress_y_2018.append(np.median(effect_2018_y[lo:hi]))
    regress_s_2018.append(np.median(effect_2018_s[lo:hi]))
    regress_dates_2018.append(temps_2018["datetime"][int((lo + hi) / 2)])
regress_temps_2018 = np.array(regress_temps_2018)
regress_y_2018 = np.array(regress_y_2018)
regress_s_2018 = np.array(regress_s_2018)
regress_dates_2018 = np.array(regress_dates_2018)
# Linear regression of Y translation.
lr_y_2018, errors_lr_y = temperature.regress_and_errors(regress_temps_2018, regress_y_2018)
print(f"Max = {np.max(errors_lr_y)}, mean = {np.mean(errors_lr_y)}, std = {np.std(errors_lr_y)}")
# Linear regression of strain.
lr_s_2018, errors_lr_s = temperature.regress_and_errors(regress_temps_2018, regress_s_2018)
print(f"Max = {np.max(errors_lr_s)}, mean = {np.mean(errors_lr_s)}, std = {np.std(errors_lr_s)}")

Max = 0.943733843632121, mean = 0.3607051428376871, std = 0.23044170970120179
Max = 5.180673260344873e-05, mean = 1.0151069228496279e-05, std = 8.72283879506276e-06


#### Setup damage scenarios.

In [11]:
damage_scenarios = [healthy_damage]
traffic_arrays = [traffic_array_0]
damage_names = ["Healthy"]
for pier_settlement in np.arange(start=0, stop=3, step=0.05):
    damage_scenarios.append(pier_disp_damage([(5, pier_settlement / 1000)]))
    traffic_arrays.append(traffic_array_0)
    damage_names.append(f"Settlement of pier 5 by {pier_settlement:.0f} mm")
response_types = [rt_y, rt_s]

#### Collect responses under each damage scenario.

In [12]:
responses = [[None for _ in response_types] for _ in damage_scenarios]
for d_i, (damage_scenario, traffic_array) in enumerate(zip(damage_scenarios, traffic_arrays)):
    for r_i, response_type in enumerate(response_types):
        responses[d_i][r_i] = responses_to_traffic_array(
            c=damage_scenario.use(c)[0],
            traffic_array=traffic_array,
            response_type=response_type,
            damage_scenario=damage_scenario,
            points=[sensor_point],
        ).T[0]  # Responses from a single point.
        # Resize responses into mm and strain.
        if response_type == rt_y:
            responses[d_i][r_i] = resize_y(responses[d_i][r_i])
        elif response_type == rt_s:
            responses[d_i][r_i] = resize_s(responses[d_i][r_i])
responses = np.array(responses)
responses.shape

(144001, 2400)
(2400, 1)
(144001, 2400)
(2400, 1)
(144001, 2400)
(2400, 1)
(144001, 2400)uilt FEMResponses in 0.01s, (ResponseType.YTranslation)
(2400, 1)
(144001, 2400)uilt FEMResponses in 0.01s, (ResponseType.Strain)
(2400, 1)
(144001, 2400)uilt FEMResponses in 0.01s, (ResponseType.YTranslation)
(2400, 1)
(144001, 2400)uilt FEMResponses in 0.01s, (ResponseType.Strain)
(2400, 1)
(144001, 2400)uilt FEMResponses in 0.01s, (ResponseType.YTranslation)
(2400, 1)
(144001, 2400)uilt FEMResponses in 0.01s, (ResponseType.Strain)
(2400, 1)
(144001, 2400)uilt FEMResponses in 0.01s, (ResponseType.YTranslation)
(2400, 1)
(144001, 2400)uilt FEMResponses in 0.01s, (ResponseType.Strain)
(2400, 1)
(144001, 2400)uilt FEMResponses in 0.01s, (ResponseType.YTranslation)
(2400, 1)
(144001, 2400)uilt FEMResponses in 0.01s, (ResponseType.Strain)
(2400, 1)
(144001, 2400)uilt FEMResponses in 0.01s, (ResponseType.YTranslation)
(2400, 1)
(144001, 2400)uilt FEMResponses in 0.01s, (ResponseType.Strain)
(2400, 1)
(

(61, 2, 144001)

#### Removal of temperature effect.

In [ ]:
downsample = 100
data = []
datetimes = []
for month_i, day_i in itertools.product([7, 8, 9, 10, 11, 12], np.arange(1, 30 + 1)):
    day_str, month_str = str(day_i), str(month_i)
    if len(day_str) == 1: day_str = "0" + day_str
    if len(month_str) == 1: month_str = "0" + month_str
    temp_i, temp_j = temperature.ij(temps_2019, f"2019-{month_str}-{day_str}T00:00", f"2019-{month_str}-{day_str}T23:59")
    day_temp = np.mean(temps_2019["temp"][temp_i:temp_j])
    print(f"Day, month = {day_str}, {month_str}, Mean day temperature = {day_temp:.3f}", end="\r")
    rm_year_s = lr_s_2018.predict([[day_temp]])[0]
    rm_year_y = lr_y_2018.predict([[day_temp]])[0]
#     print(f"Y/S effect from 2018 = {rm_year_y:.3f}, {rm_year_s}")
    datetimes.append(datetime.fromisoformat(f"2019-{month_str}-{day_str}T00:00"))
    data.append([])
    for (traffic_y, traffic_s), damage_name in zip(responses, damage_names):
        temp_y = temperature.apply(effect=effect_2019_y[temp_i:temp_j], responses=traffic_y)
        temp_s = temperature.apply(effect=effect_2019_s[temp_i:temp_j], responses=traffic_y)
        _rm_24h_y, rm_24h_y = temperature.remove_sampled(24, traffic_y + temp_y)
        _rm_24h_s, rm_24h_s = temperature.remove_sampled(24, traffic_s + temp_s)
        signal_s = traffic_s + temp_s - rm_24h_s - rm_year_s
        signal_y = traffic_y + temp_y - rm_24h_y - rm_year_y
#         signal_s = traffic_s + temp_s - rm_year_s
#         signal_y = traffic_y + temp_y - rm_year_y
        data[-1].append(np.array([signal_y, signal_s]))
for i in range(len(data)):
    data[i] = np.array(data[i])
# data = np.resize(data, (data.shape[1], data.shape[2], data.shape[0], data.shape[3]))
print()
len(data), data[0].shape

#### 6 month daily prediction

In [ ]:
results_daily = []
for scen_i in range(data[0].shape[0]):
    results_daily.append([])
    for day_i in range(len(data)):
        results_daily[-1].append(np.mean(data[day_i][scen_i][0]))
results_daily = np.array(results_daily)
# plt.plot(results_daily[0])
results_daily.shape

#### 6 month 14 day prediction

In [ ]:
days = 14  # Number of days for robust estimator.
results_robust = []
for scen_i in range(len(results_daily)):
    results_robust.append([])
    for day_i in range(days, len(data)):
        results_robust[-1].append(np.mean(results_daily[scen_i][day_i-days:day_i]))
results_robust = np.array(results_robust)
results_robust.shape
plt.plot(results_robust[0])
plt.plot(results_robust[-1])

In [ ]:
threshold_step = -0.01
thresholds = np.arange(start=1, stop=-3, step=threshold_step)
counts = []  # Indexed by threshold then line/pier settlement.
for threshold in thresholds:
    # For each threshold, maintain a count for each line/pier settlement.
    counts.append([0 for _ in range(results_robust.shape[0])])
    for day_i in range(results_robust.shape[1]):
        # For each day, check if each line is below threshold.
        for line_i in range(results_robust.shape[0]):
            if results_robust[line_i][day_i] < threshold:
                counts[-1][line_i] += 1
counts = np.array(counts).T
counts.shape

In [ ]:
# Start by plotting for one pier settlement.
count_hs, count_ps = counts[0], counts[2]

tp, fn = count_ps, results_robust.shape[1] - count_ps
tn, fp = results_robust.shape[1] - count_hs, count_hs

plt.plot(thresholds, (tp) / (fn + tp), label="Sensitivity", color="tab:green", lw=4)
plt.plot(thresholds, (tn) / (fp + tn), label="Specificity", color="tab:blue", lw=4)
plt.plot(thresholds, (fn) / (fn + tp), label="Miss rate", color="tab:orange", lw=4)
plt.plot(thresholds, (fp) / (fp + tn), label="False positive rate", color="tab:red", lw=4)
plt.plot(thresholds, (tn + tp) / (fn + fp + tn + tp), label="Accuracy", color="black", lw=4)
plt.legend()
plt.title(damage_names[2])

#### Plot accuracy as a function of pier settlement

In [ ]:
accuracies = []
settlements = []
for pier_i, pier_settlement in enumerate(damage_scenarios[1:]):
    pier_settlement = pier_settlement.pier_disps[0].displacement * 1000
    settlements.append(pier_settlement)
    count_hs, count_ps = counts[0], counts[pier_i + 1]
    tp, fn = count_ps, results_robust.shape[1] - count_ps
    tn, fp = results_robust.shape[1] - count_hs, count_hs
    accuracy = (tn + tp) / (fn + fp + tn + tp)  # For each threshold.
    accuracies.append(sum(1 if a >= 1 else 0 for a in accuracy) * -threshold_step)
plt.plot(settlements, accuracies)
plt.ylabel("Range of 100% accurate sensor thresholds")
plt.xlabel("Settlement of pier 5 by (mm)")
plt.title("Accurate sensor thresholds as a function of pier settlement")
plt.savefig(os.path.join(IMAGE_PATH, f"sensor-acc-{sensor_point.x:.0f}-{sensor_point.z:.0f}.pdf"))